<a href="https://colab.research.google.com/github/InTEGr8or/jupyter-fun/blob/master/nCov2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infection Rates per day


## Purpose and Source

[this code available on Github](https://github.com/InTEGr8or/jupyter-fun/blob/master/nCov19.ipynb)

The percent rates in the sheet at the bottom are approximations. Data seems to be released about twice a day. Sometimes there is a lag, and they release it at different times.

Optimally, the time of the update would be taken into account and prorated per hour and multiplied by the number of hours difference but I am just starting to learn Python so there are probably a lot of improvements that could be made.

Data is collected by Johns Hopkins in Baltimore and published here: [nCov19 contagion](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)

Here is another excellent data presentation at [WorldOMeters.info](https://www.worldometers.info/coronavirus/#repro)

Now a [Time Series Table](https://docs.google.com/spreadsheets/u/1/d/1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo/htmlview?usp=sharing&sle=true) is available, and a [Feature Layers](https://gisanddata.maps.arcgis.com/home/item.html?id=c0b356e20b30490c8b8b4c7bb9554e7c) appears to be available but it requires authentication.

The single-sheet Time Series is a much cleaner data source and _doesn't require repeated reauthentication_ so I'm reworking it to use that and we don't have a percent change right now, until I figure out how to use Pandas properly.

## This first section sets up the imports and some parsing functions.


In [0]:
try:
  from bs4 import BeautifulSoup
except:
  !pip install beautifulsoup4
  from bs4 import BeautifulSoup
  
import requests
import numpy as np
from dateutil import parser
from datetime import datetime
import pandas as pd

tsc_csv = "https://docs.google.com/spreadsheets/u/1/d/1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo/export?format=csv"
tsc_html = "https://docs.google.com/spreadsheets/u/1/d/1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo/htmlview?usp=sharing&sle=true#"

pd.set_option('display.max_rows', 200)

states = {
  'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa',
  'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut',
  'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
  'GU': 'Guam', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois',
  'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts',
  'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota',
  'MO': 'Missouri', 'MP': 'Northern Mariana Islands', 'MS': 'Mississippi',
  'MT': 'Montana', 'NA': 'National',
  'NC': 'North Carolina',
  'ND': 'North Dakota',
  'NE': 'Nebraska',
  'NH': 'New Hampshire',
  'NJ': 'New Jersey',
  'NM': 'New Mexico',
  'NV': 'Nevada',
  'NY': 'New York',
  'OH': 'Ohio',
  'OK': 'Oklahoma',
  'OR': 'Oregon',
  'PA': 'Pennsylvania',
  'PR': 'Puerto Rico',
  'RI': 'Rhode Island',
  'SC': 'South Carolina',
  'SD': 'South Dakota',
  'TN': 'Tennessee',
  'TX': 'Texas',
  'UT': 'Utah',
  'VA': 'Virginia',
  'VI': 'Virgin Islands',
  'VT': 'Vermont',
  'WA': 'Washington',
  'WI': 'Wisconsin',
  'WV': 'West Virginia',
  'WY': 'Wyoming',
  'AB': 'Alberta',
  'BC': 'British Columbia',
  'MB': 'Manitoba',
  'NB': 'New Brunswick',
  'NL': 'Newfoundland and Labrador',
  'NT': 'Northwest Territories',
  'NS': 'Nova Scotia',
  'NU': 'Nunavut',
  'ON': 'Ontario',
  'PE': 'Prince Edward Island',
  'QC': 'Quebec',
  'SK': 'Saskatchewan',
  'YT': 'Yukon'
}

def is_date(text):
  try:
    s = parser.parse(text)
    return True
  except:
    return False

def hotten(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    if '%' in s:
      # print(s)
      is_max = float(s.replace('%', '')) > 20
      return ['background-color: yellow' if v else '' for v in is_max]
    else:
      return 'background-color: black'

def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    heat = str(hex(min(int(val.replace('%', '')) * 10 + 56, 255))).split('x')[-1].upper()
    color = f'#{heat}5555'
    # print(val, heat, color)
    return 'color: %s' % color

## Get Data, Loop Through and Print

This section is massively simplified with now that Johns Hopkins tidied up their data sources (Thanks Johns Hopkins tech people! _Way_ better)

I'm sure they have a lot of task on their hands and it's nice to see they can retrace over their first crack at the organization of the data that was coming in.

In [31]:
dft = pd.read_csv(tsc_csv)

dates = [col for i, col in enumerate(dft.columns) if is_date(col) ]
dates.reverse()

dfr = pd.read_csv(tsc_csv + "&gid=1940183135")
dfm = pd.read_csv(tsc_csv + "&gid=1056055583")
dfm.rename(columns={'Country/Region': 'Country', 'Province/State': 'State'}, inplace=True)
dfm['Country'].replace('Mainland China','China', inplace=True)

# Get date of first death or days since first death.
for i, row in dfm.iterrows():
  mdates = [col for i, col in enumerate(dft.columns) if is_date(col)]
  for date in mdates:
    if row[date] > 0:
      print('Death Date:', row['Country'], row['State'], date, row)
      row['First Death Date'] = date

dfm.apply
dfm.set_index(['Country', 'State'], inplace=True)
dft.fillna('')
df = pd.DataFrame()


# Get tabs from Html
page = requests.get(tsc_html)
bs = BeautifulSoup(page.text)
tpath = "//div[@id='sheets-viewport']/div //table"
tables = bs.select('div#sheets-viewport div table')
trc = tables[0].select("tr")
th = trc[0].select('th')
# print(tables[1])

# First confirmed date in country (Est.)
df['Country'] = dft['Country/Region'].replace('Mainland China', 'China').replace('United Arab Emirates', 'UAE')
df['State'] = dft['Province/State'].fillna('')
# df.set_index(['Country', 'State'], inplace=True)
df['First Case'] = dft['First confirmed date in country (Est.)']
df['Days Ago'] = datetime.now() - df['First Case'].apply(lambda date: parser.parse(date, fuzzy=True))
df['Days Ago'] = df['Days Ago'].apply(lambda days: str(days).split(' ')[0])
df['Death Toll'] = 0
for i, row in df.iterrows():
  # if ',' in row['State']:
  #   print(row['State'], states[row['State'].split(',')[1].strip()])
  if row['State'] == '':
    try:
      mrow = dfm.loc[row['Country'], dates[0]]
      if 'Phil' in row['Country']:
        print('Phil:', row['Country'], type(mrow))
    except:
      mrow = 0
  else:
    try:
      mrow = dfm.loc[row['Country'], :].loc[row['State'].split(',')[0], dates[0]]
    except:
      mrow = 0
  if isinstance(mrow, (int, float, complex)) and not isinstance(mrow, bool):
    if mrow > 0:
      # print(row['Country'], row['State'], mrow)
      df.at[i, 'Death Toll'] = mrow

# Remove early results
for i, date in enumerate(dates):
  if i < len(dates) -1 and parser.parse(date).day == parser.parse(dates[i + 1]).day:
    del dates[i + 1]
# Append to columns
percents = []
for i, date in enumerate(dates):
  d = parser.parse(date)
  col = d.strftime('%B')[:3] + d.strftime('%d')
  df[col] = dft[date].replace(np.inf, 0).fillna(0).astype(int)
  if i < len(dates) - 1:
    pcol = dates[i + 1]
    percents.append(col + '%')
    df[col + '%'] = round((dft[date] / dft[dates[i + 1]].fillna(0) * 100) - 100).replace(np.inf, 0).fillna(0).astype(int).astype(str) + '%'


Death Date: China Beijing 1/27/2020 7:00 PM State                               Beijing
Country                               China
First confirmed date in country    1/3/2020
Lat                                 40.1824
Long                                116.414
1/21/2020 10:00 PM                      NaN
1/22/2020 12:00 PM                      NaN
1/23/2020 12:00 PM                      NaN
1/24/2020 12:00 AM                      NaN
1/24/2020 12:00 PM                      NaN
1/25/2020 12:00 AM                      NaN
1/25/2020 12:00 PM                      NaN
1/25/2020 10:00 PM                      NaN
1/26/2020 11:00 AM                      NaN
1/26/2020 11:00 PM                      NaN
1/27/2020 9:00 AM                       NaN
1/27/2020 7:00 PM                         1
1/27/2020 8:30 PM                         1
1/28/2020 1:00 PM                         1
1/28/2020 6:00 PM                         1
1/28/2020 11:00 PM                        1
1/29/2020 1:30 PM               

## Render

In [24]:
# df.set_index('Country', inplace=True)
df.drop(columns=['First Case']).sort_values(by=['Country','State'], ascending=[True, True]).style.applymap(color_negative_red, subset=percents)

,Country,State,Days Ago,Death Toll,Feb07,Feb07%,Feb06,Feb06%,Feb05,Feb05%,Feb04,Feb04%,Feb03,Feb03%,Feb02,Feb02%,Feb01,Feb01%,Jan31,Jan31%,Jan30,Jan30%,Jan29,Jan29%,Jan28,Jan28%,Jan27,Jan27%,Jan26,Jan26%,Jan25,Jan25%,Jan24,Jan24%,Jan23,Jan23%,Jan22,Jan22%,Jan21
49,Australia,New South Wales,13,0,4,0%,4,0%,4,0%,4,0%,4,0%,4,0%,4,0%,4,0%,4,0%,3,0%,3,0%,3,0%,3,0%,0,0%,0,0%,0,0%,0,0%,0
51,Australia,Queensland,12,0,5,25%,4,0%,4,33%,3,50%,2,0%,2,0%,2,0%,2,0%,2,100%,1,0%,1,0%,1,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0
66,Australia,South Australia,7,0,2,0%,2,0%,2,0%,2,0%,2,0%,2,0%,2,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0
50,Australia,Victoria,13,0,4,0%,4,0%,4,0%,4,0%,4,0%,4,0%,4,33%,3,0%,3,50%,1,0%,1,0%,1,0%,1,0%,0,0%,0,0%,0,0%,0,0%,0
69,Belgium,,4,0,1,0%,1,0%,1,0%,1,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0
52,Cambodia,,12,0,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0
46,Canada,British Columbia,11,0,4,100%,2,0%,2,100%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0
59,Canada,"London, ON",8,0,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,1,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0,0%,0
45,Canada,"Toronto, ON",13,0,2,0%,2,0%,2,0%,2,0%,2,0%,2,0%,2,0%,2,100%,1,0%,1,0%,1,0%,1,0%,1,0%,0,0%,0,0%,0,0%,0,0%,0
0,China,Anhui,36,0,733,24%,591,0%,591,12%,530,10%,480,18%,408,37%,297,25%,237,18%,200,0%,152,0%,106,0%,70,0%,70,17%,39,160%,15,67%,9,800%,1,0%,0
